# Código de Testes
-----

Use esse código para testar a trajetória da bola para pontos específicos

#### Bibliotecas e Funções Básicas

In [10]:
# Importa bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.integrate import odeint

In [11]:
# Função para deixar os eixos 3D igual (equivalente ao plt.axes('equal'))
def set_axis_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)
    
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([0, z_middle + plot_radius])
# Fonte: https://stackoverflow.com/questions/13685386/matplotlib-equal-unit-length-with-equal-aspect-ratio-z-axis-is-not-equal-to

#### Parâmetros

In [12]:
d_ar = 1.2 # Densidade do Ar [kg/m³]
g = 9.8 # Aceleração da Gravidade [m/s]

cl = 1 # Coeficiente de Lift (Magnus)
cd = 0.1 # Coeficiente de Arrasto

r = 0.105 # Raio da Bola [m]
m = 0.27 # Massa da Bola [kg]
A = pi*(r**2) # Álea Transversal da Bola [m²]

#### A Quadra de Volei

In [13]:
# Quadra: 9 mestros de largula por 18 metros de comprimento
# Rede: 2,43 metros de altura, localizada no meio da quadra (9 metros)

quadra_x = [0, 18, 18, 0, 0]
quadra_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
quadra_z = [0, 0, 0, 0, 0]

rede_x = [9, 9, 9, 9, 9]
rede_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
rede_z = [1.43, 2.43, 2.43, 1.43, 1.43]

#### Declarações iniciais

In [14]:
# Ângulos iniciais
tetha = radians(90) # Ângulo vertical
alpha = radians(15) # Ângulo horizontal

# Posição inicial
x0 = 0
y0 = -4
z0 = 3 # Nesse sistema, a altura está no eixo Z

# Decompondo a Velocidade inicial
v0 = 126.3/3.6 # dado em [km/h] transformado em [m/s]
vx0 = v0*sin(tetha)*cos(alpha)
vy0 = v0*sin(tetha)*sin(alpha)
vz0 = v0*cos(tetha)

# Decompondo a Velocidade Angular inicial
w = 7*(2*pi) # dado em [RPS] transformado em [rad]
wx = -w*sin(tetha)*sin(alpha)
wy = w*sin(tetha)*cos(alpha)
wz = w*cos(tetha)
w_vetor = np.array([wx, wy, wz]) # Compondo o Vetor da velocidade angular

# Valores iniciais
info_0 = [x0, y0, z0, vx0, vy0, vz0]

# Definindo o Tempo
dt = 1e-3
tMax = 1.5+dt
t_lista = np.arange(0, tMax, dt)

#### Modelo

In [15]:
def modelo(info, t_lista):
    # Extraindo os Valores
    x = info[0]
    y = info[1]
    z = info[2]
    vx = info[3]
    vy = info[4]
    vz = info[5]
    
    v_vetor = np.array([vx, vy, vz]) # Compondo o vetor da velocidade
    
    v_xy = sqrt(vx**2 + vy**2) # Componente da velocidade em xy
    v = sqrt(v_xy**2 + vz**2) # Velocidade

    # Calculos dos ângulos
    cos_tetha = vz/v 
    sen_tetha = v_xy/v
    cos_alpha = vx/v_xy
    sen_alpha = vy/v_xy

    # Calculando a Força de Arrasto
    Fa = 0.5*d_ar*A*cd*(v**2)
    Fa_x = -Fa*sen_tetha*cos_alpha
    Fa_y = -Fa*sen_tetha*sen_alpha
    Fa_z = -Fa*cos_tetha
    
    # Calculando a Força Magnus
    Fmag = 0.5*d_ar*pi*(r**3)*cl*np.cross(w_vetor, v_vetor)
    Fmag_x = Fmag[0]
    Fmag_y = Fmag[1]
    Fmag_z = Fmag[2]
    
    # Calculando a aceleração (dV_dt)
    dVx_dt = (Fa_x + Fmag_x)/m
    dVy_dt = (Fa_y + Fmag_y)/m
    dVz_dt = (-(m*g) + Fa_z + Fmag_z)/m

    if z <= 0:
        return [0, 0, 0, 0 ,0 ,0]
    else: 
        return [vx, vy, vz, dVx_dt ,dVy_dt ,dVz_dt]    

#### Teste

In [16]:
# Implementando
infos_Odeint = odeint(modelo, info_0, t_lista)

posX = list(infos_Odeint[:,0])
posY = list(infos_Odeint[:,1])
posZ = list(infos_Odeint[:,2])

index = len(t_lista)-1
rede = False
cor = "red"
for i, t in enumerate(t_lista):
    if not rede and (posX[i] >= 9 and posZ[i]-r >= 2.43):
        print(f'tempo para passar da rede: {t}s')
        rede = True
        cor = "orange"
    if rede and (posZ[i] <= 0 and posX[i] <= 18):
        print(f'tempo para bater no chão: {t}s')
        cor = "green"
        index = i
        break
    
print(f'posição final em X: {round(posX[index], 2)}m')
print(f'posição final em Y: {round(posY[index], 2)}m')
print(f'posição final em Z: {round(posZ[index], 2)}m')

posição final em X: 16.85m
posição final em Y: 0.52m
posição final em Z: -0.0m


In [17]:
%matplotlib inline
%matplotlib qt5

ax = plt.axes(projection='3d')
ax.plot3D(rede_x, rede_y, rede_z, color='grey') # rede
ax.plot3D(quadra_x, quadra_y, quadra_z, color='black') # quadra
ax.plot3D(posX[0], posY[0], posZ[0], 'ro') # Bola

ax.plot3D(posX[0:index], posY[0:index], posZ[0:index], color=cor)
plt.title("Trajetória da Bola de Volei (Teste)")
plt.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend()
set_axis_equal(ax)
plt.show()

No handles with labels found to put in legend.


In [18]:
# # ANIMAÇÃO
# plt.close('all')
# %matplotlib qt5
# get_ipython().magic('matplotlib qt5')
# import matplotlib.patches as patches
# from matplotlib import animation

# fig = plt.figure()
# plt.title('Saque de Volei')
# plt.xlabel('X')
# plt.ylabel('Z')
# #plt.axis('equal')

# ax = fig.add_subplot()

# ax.set_xlim(-0.1, 20)
# ax.set_ylim(-0.1, 5) # Eixo Z

# bola = patches.Circle((x0, z0), r, fc=cor)
# rede = patches.Rectangle((9, 1.43), 0.1, 1, fc='gray')
# quadra = patches.Rectangle((0, -0.1), 18, 0.1, fc='black')

# def init():
#     ax.add_patch(bola)
#     ax.add_patch(rede)
#     ax.add_patch(quadra)
#     return None

# def animate(i):    
#     bola.center = posX[i], posZ[i]
#     return None

# anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(t_lista), interval=1, blit=False)